# ML Pipeline testing
---
---

# Imports

#### Standard library imports

In [1]:
import sys
import pickle
import os

sys.path.append("../../")

#### Third party imports

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)

#### Local application imports

In [3]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

# Base dataset with no metadata

In [ ]:
## Function parameters
db_crds = 'pc_db_prod'
query = '''
SELECT cita.citaid as appointment_id,
       citafecha as appointment_date,
       citahorad as appointment_start_time,
       citahorah as appointment_end_time,
       citaestado as appointment_status,
       u.usuarionomfull as doctor,
       e.especialidadnom as medical_specialty,
       su.sucursalnom as clinic,
       se.servicionom as service,
       p.pacienteid as patient_id,
       p.pacientefnac as patient_birth_date

FROM cita

 INNER JOIN usuario u ON cita.citadoctorid = u.usuarioid
 INNER JOIN servicio se ON cita.servicioid = se.servicioid
 INNER JOIN especialidad e ON u.usuarioespecialidadid = e.especialidadid
 INNER JOIN sucursal su ON cita.citasucursalid = su.sucursalid
 LEFT JOIN paciente p ON cita.pacienteid = p.pacienteid

WHERE citafecha >= '2022-01-01'
  AND citafecha <= '2023-12-31'

;
'''


In [ ]:
dfx = sql_string_to_df(db_crds, query)
dfx.columns = ['appointment_id', 'appointment_date', 'appointment_start_time', 'appointment_end_time', 'appointment_status', 'doctor', 'medical_specialty', 'clinic', 'service', 'patient_id', 'patient_birth_date']

### Formatting date columns

In [ ]:
dataset

In [ ]:
## Formatting appointment date
dataset['appointment_date'] = pd.to_datetime(dataset['appointment_date'], format='%Y-%m-%d')

### Appointment status simplification

In [ ]:
## Reference to simplify labels in appointment status
values_map = {

    ## Completed
    'COMPLETADA': 'completed',
    'SALA_ESPERA': 'completed',
    'CONSULTA': 'completed',
    'CONFIRMADO_PAGO': 'completed',

    ## No shows
    'NO_PRESENTO': 'no_show',
    'MENSAJE_DEJADO': 'no_show',
    'NO_CONFIRMADO': 'no_show',
    'CONFIRMADO': 'no_show',
    'LLEGA_TARDE_ESPERA': 'no_show',
    'REAGENDAR': 'no_show',
    'TRIAGE': 'no_show',
    'TRIAGE_COMPLETO': 'no_show',
    'LISTA_ESPERA': 'no_show',
    'VALIDA_DATOS': 'no_show',

    ## Unused
    'DISPONIBLE': 'unused',

    ## Blocked
    'BLOQUEADO': 'blocked',

    ## Canceled
    'CANCELA_PACIENTE': 'cancel_patient',
    'CANCELA_EMPLEADO': 'cancel_employee',
    
    
}

In [ ]:
dataset['appointment_status'].value_counts()

In [ ]:
## Mapping values according to reference
dataset['appointment_status_simplified'] = dataset['appointment_status'].map(values_map).fillna(dataset['appointment_status'])

In [ ]:
dataset['appointment_status_simplified'].value_counts()

In [ ]:
dataset.drop(['appointment_status'], axis=1, inplace=True)

### Saving data

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'xxx'

pickle.dump(
    dataset,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [5]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'xxx'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dataset = pickle.load(obj_content)

In [ ]:
## Saving as csv
dataset.to_csv('../../pkg_dir/data/pickles/robs/dataset.csv')

# Metadata actions

### Extracting from SQL

In [ ]:
sql_string = """
SELECT cita.citaid as appointment_id,
       c.cestatusobs as metadata,
       SUBSTRING(c.cestatusobs, '^[^ ]*') as metadata_usr,
       SUBSTRING(c.cestatusobs, '\((.+?)\)') as metadata_datetime,
       SUBSTRING(c.cestatusobs, ' :(.*?)  ') as metadata_action,
       SUBSTRING(c.cestatusobs, ' :(.*)') as metadata_action_full

FROM cita

 INNER JOIN citaestatus c ON cita.citaid = c.citaid AND cita.citaanio = c.citaanio


WHERE citafecha >= '2022-01-01'
  AND citafecha <= '2023-12-31'

;
"""

In [ ]:
dfx = sql_string_to_df('pc_db_prod', sql_string)

In [ ]:
dfx

In [ ]:
dfx.rename(
    columns={
        0: "appointment_id",
        1: "metadata_full",
        2: "user",
        3: "date",
        4: "action",
        5: "action_full",
    }, 
    inplace=True
)

### Pickle save/load

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'metadata_raw.pkl'

pickle.dump(
    dfx,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'metadata_raw.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfx = pickle.load(obj_content)

### Cleaning actions fields

In [ ]:
dfz = dfx.copy()

In [ ]:
## Dummy col
dfz['action_clean'] = 'irrelevant'

In [ ]:
### Confirmado
mr1 = dfz['action_full'].str.contains('Confirmado')

dfz.loc[mr1, 'action_clean'] = 'confirmed'

In [ ]:
### Creado
mr1 = dfz['action_full'].str.contains('Creado')

dfz.loc[mr1, 'action_clean'] = 'created'

In [ ]:
### Sala espera
mr1 = dfz['action_full'].str.contains('En sala de espera')

dfz.loc[mr1, 'action_clean'] = 'in_waiting_room'

In [ ]:
### Completado
mr1 = dfz['action_full'].str.contains('Completado')
mr2 = dfz['action_full'].str.contains('Completada')
mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'completed'

In [ ]:
### En consulta
mr1 = dfz['action_full'].str.contains('En consulta')

dfz.loc[mr1, 'action_clean'] = 'with_doctor'

In [ ]:
### No se presentó
mr1 = dfz['action_full'].str.contains('No se presentó')

dfz.loc[mr1, 'action_clean'] = 'no_show'

In [ ]:
### Cancelada por el paciente
mr1 = dfz['action_full'].str.contains('Cancelada por el paciente')

dfz.loc[mr1, 'action_clean'] = 'canceled_by_patient'

In [ ]:
### Cancelada por empledo
mr1 = dfz['action_full'].str.contains('Cancelada por empledo')

dfz.loc[mr1, 'action_clean'] = 'canceled_by_employee'

In [ ]:
### Se actualiza la fecha/hora
mr1 = dfz['action_full'].str.contains('Se actualiza la fecha/hora')

dfz.loc[mr1, 'action_clean'] = 'update_date_hour'

In [ ]:
### Disponible
mr1 = dfz['action_full'].str.contains('Disponible')

dfz.loc[mr1, 'action_clean'] = 'available'

In [ ]:
### Mensaje dejado
mr1 = dfz['action_full'].str.contains('Mensaje dejado')

dfz.loc[mr1, 'action_clean'] = 'message_left'

### Formatting and adjusting

In [ ]:
dfz[dfz['action_clean'] == 'completed']

In [ ]:
## Dropping irrelevant columns
mr1 = dfz['action_clean'] != 'irrelevant'
rc = ['appointment_id', 'user', 'date', 'action_clean']

dfz = dfz.loc[mr1, rc].copy()

In [ ]:
## Adjusting dateformat
dfz['date'] = pd.to_datetime(dfz['date'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

dfz = dfz.loc[~dfz['date'].isna(), :].copy()

In [ ]:
## Sorting values 
dfz.sort_values(by=['appointment_id', 'action_clean', 'date'], inplace=True, ascending=False)

In [ ]:
## Keeping latest 
dfz.drop_duplicates(keep='first', subset=['appointment_id', 'action_clean'], inplace=True)

### Pivot

In [ ]:
dfp = dfz.pivot(index='appointment_id', columns='action_clean', values=['user', 'date'])

In [ ]:
dfp.columns = ['meta__' + col[1] + '_$' + col[0] for col in dfp.columns]

In [ ]:
cols = dfp.columns.tolist()
cols.sort()

dfp = dfp.loc[:, cols].copy()

### Pickle save/load

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'pivot_df_v2.pkl'

pickle.dump(
    dfp,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'pivot_df_v2.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfp = pickle.load(obj_content)


### Merging pivot dataframe and main

In [ ]:
dfp.reset_index(inplace = True)

In [ ]:
dataset = pd.merge(dfx, dfp, on='appointment_id', how='left')

# Final dataset

In [4]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'dataset.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dataset = pickle.load(obj_content)

# 

---

---